In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("youtube_dataset.csv")

In [3]:
df.head(5)

,artist,song,text,youtube_url
0,Eric Clapton,Don't Cry Sister,"do n't cri sister cri , it 'll be alright in t...",https://www.youtube.com/watch?v=8WZXFlqdE1s
1,George Strait,Check Yes Or No,it start way back in third grade i use to sit ...,https://www.youtube.com/watch?v=NHxS8wlDngI
2,Coldplay,Such A Rush,such a rush to do noth at all such a fuss to d...,https://www.youtube.com/watch?v=5bMzeIqiBBY
3,Phil Collins,I Wish It Would Rain Down,you know i never meant to see you again but i ...,https://www.youtube.com/watch?v=YcY3FH208l8
4,Elton John,I Must Have Lost It On The Wind,i 'm no longer count i 'm not keep score i cou...,https://www.youtube.com/watch?v=-9dJy5TIRYE


In [4]:
df.tail(5)

,artist,song,text,youtube_url
4995,Annie,Little Girls,miss hannigan littl girl littl girl everywher ...,NaN
4996,Pitbull,Hey You Girl,ha ha ha yo it that left field thinkin ' out t...,NaN
4997,Bob Rivers,Read It In The Tabloids,i bet you 're wonder how i knew that princ cha...,NaN
4998,Ramones,I Remember You,i rememb you i rememb you i rememb you i remem...,NaN
4999,Michael Jackson,Ease On Down The Road,"come on and eas on down , eas on down the road...",NaN


In [5]:
df.shape

(5000, 4)

In [6]:
df.isnull().sum()

artist            0
song              0
text              0
youtube_url    4900
dtype: int64

In [7]:
df =df.sample(5000).drop('youtube_url', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Pink Floyd,Lucy Leave,"leav , when i ask you to leav , luci pleas , f..."
1,Pretenders,I'll Stand By You,"oh , whi you look so sad , tear are in your ey..."
2,Aerosmith,Permanent Vacation,i got a letter from a friend the other morn he...
3,Avril Lavigne,Give It Up,everytim you go away it actual kinda make my d...
4,Culture Club,Black Money,do you deal in black money ? do you deal in bl...
5,Judds,Mama He's Crazy,"mama , i found someon like you said would come..."
6,Justin Bieber,One Dance (Remix),[ intro : kyla ] babi i like your style [ vers...
7,Kim Wilde,Back Street Joe,danc with back street joe that wa a time i lov...
8,Point Of Grace,Free Indeed,can you think of the way that we use to be we ...
9,Rascal Flatts,I'll Be Home For Christmas,i 'll be home for christma you can count on me...


In [9]:
df['text'][0]

"leav , when i ask you to leav , luci pleas , far away from me , luci oh , go littl girl seen , is ( oh so ) broken up about you , luci mean , treat me and done me harm , luci been , in love with you and your charm , luci oh , go littl girl i 'm in love with you , luci you got my heart you got my heart , oh no ! you tear me apart you just wo n't let me go you hold on so tight , so tight i just ca n't breath now luci leav , luci leav , when i ask you to leav , littl girl pleas , far away from me , littl girl yeah ! go littl girl seen , is ( oh so ) broken up about you , luci yeah ! go ."

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [14]:
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer

tokenizer = TreebankWordTokenizer()
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = tokenizer.tokenize(txt)  # Avoids the broken sent_tokenize route
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)


In [15]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/srivastava/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
df['text'] = df['text'].apply(tokenization)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [19]:
similarity[0]

array([1.        , 0.02818125, 0.03210611, ..., 0.10256227, 0.01659779,
       0.02870928], shape=(5000,))

In [20]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [21]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [23]:
recommendation('Lucy Leave')

['Rubber Lucy',
 "Lucy Can't Dance",
 "I'm Not A Stranger",
 'Lucy In The Sky With Diamonds',
 'Edge Of The Moon',
 'White Lightening',
 'Land Of 1000 Dances',
 'Oh Girl',
 'Out Of Town Girl',
 'Hey Little Girl',
 'Little Girls',
 'Leaving Here',
 "She's Leaving You",
 'A Little Bit Of Green',
 'Just A Little Bit Of Your Heart',
 "Don't Leave Me Where You Found Me",
 'Hand On My Heart',
 'Go On Girl',
 'One Time',
 'For The Girl Who Has Everything']

In [24]:
import pickle
pickle.dump(similarity,open('similarity_yt.pkl','wb'))
pickle.dump(df,open('df_yt.pkl','wb'))